In [4]:
'''
영화 추천 시스템 알고리즘
 - 추천 대상자 : Toby   
 - 유사도 평점 = 미관람 영화평점 * Toby와의 유사도
 - 추천 영화 예측 = 유사도 평점 / Toby와의 유사도
'''

import pandas as pd

# 1. 데이터 가져오기 
ratings = pd.read_csv('C:\/Users/hyebin/Desktop/study/python_ML/data/movie_rating.csv')
#print(ratings) #  평가자[critic]   영화[title]  평점[rating]

In [9]:
# reset_index() : default index 삽입(index -> 첫번째 칼럼)  
# title 칼럼 지정 

# 2. pivot table 작성 : row(영화제목), column(평가자), cell(평점)
print('movie_ratings')
movie_ratings =pd.pivot_table(ratings,
                              index="title",     # => 행
                              columns="critic",
                              values="rating").reset_index()
movie_ratings # default index 추가 

# [추가] 결측치가 포함된 관측치 제거 : DF.dropna()
#print(movie_ratings.dropna())

movie_ratings


critic,title,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
0,Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
1,Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
2,Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
3,Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
4,The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
5,You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


In [11]:
# 3. 사용자 유사도 계산(상관계수 R) : 번호 행 추가 효과 
sim_users = movie_ratings.corr().reset_index() # corr(method='pearson')
sim_users # default index 추가 

critic,critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
0,Claudia Puig,1.000000,0.314970,0.028571,0.566947,0.566947,0.893405
1,Gene Seymour,0.314970,1.000000,0.963796,0.396059,0.411765,0.381246
2,Jack Matthews,0.028571,0.963796,1.000000,0.747018,0.211289,0.662849
3,Lisa Rose,0.566947,0.396059,0.747018,1.000000,0.594089,0.991241
4,Mick LaSalle,0.566947,0.411765,0.211289,0.594089,1.000000,0.924473
5,Toby,0.893405,0.381246,0.662849,0.991241,0.924473,1.000000


In [13]:
# 4. Toby 미관람 영화 추출  
# 1) movie_ratings table에서 title, Toby 칼럼으로 subset 작성 
toby_rating = movie_ratings[['title', 'Toby']] # index 칼럼 추가 
toby_rating


critic,title,Toby
0,Just My Luck,NaN
1,Lady in the Water,NaN
2,Snakes on a Plane,4.5
3,Superman Returns,4.0
4,The Night Listener,NaN
5,You Me and Dupree,1.0


In [14]:
print(toby_rating.columns) # Index(['title', 'Toby']

Index(['title', 'Toby'], dtype='object', name='critic')


In [16]:
# 칼럼명 교체 
toby_rating.columns=['title', 'rating']
toby_rating

,title,rating
0,Just My Luck,NaN
1,Lady in the Water,NaN
2,Snakes on a Plane,4.5
3,Superman Returns,4.0
4,The Night Listener,NaN
5,You Me and Dupree,1.0


In [18]:
# 2) Toby 미관람 영화제목 추출 : rating null 조건으로 title 추출 
# 형식) DF.추출칼럼[DF.조건칼럼.isnull()]
toby_not_see = toby_rating.title[toby_rating.rating.isnull()] 
toby_not_see

0          Just My Luck
1     Lady in the Water
4    The Night Listener
Name: title, dtype: object

In [19]:
print(type(toby_not_see)) # Series
toby_not_see = list(toby_not_see) # Series -> list

# list 값 포함 유무 리턴 
'''
title='Just My'
print(title in toby_not_see) # True
'''


<class 'pandas.core.series.Series'>


In [21]:
# 3) raw data에서 Toby 미관람 영화 subset 생성 
# 특정 칼럼 내용으로 subset 생성 : df[df.column.isin([list원소])]
rating_t = ratings[ratings.title.isin(toby_not_see)]
rating_t

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
4,Jack Matthews,The Night Listener,3.0
5,Mick LaSalle,Lady in the Water,3.0
7,Mick LaSalle,Just My Luck,2.0
10,Mick LaSalle,The Night Listener,3.0
12,Claudia Puig,Just My Luck,3.0
15,Claudia Puig,The Night Listener,4.5
16,Lisa Rose,Lady in the Water,2.5
18,Lisa Rose,Just My Luck,3.0
20,Lisa Rose,The Night Listener,3.0


In [22]:
# 5. Toby 미관람 영화 + Toby 유사도 join
# 1) Toby 유사도 추출 
toby_sim = sim_users[['critic','Toby']]
#help(pd.merge)
# on='critic' : critic 칼럼으로 DF 병합(join)

In [28]:
# 2) 평가자 기준 병합  
rating_t = pd.merge(rating_t, toby_sim, on='critic')
rating_t

,critic,title,rating,Toby_x,Toby_y,Toby_x,Toby_y,Toby
0,Jack Matthews,Lady in the Water,3.0,0.662849,0.662849,0.662849,0.662849,0.662849
1,Jack Matthews,The Night Listener,3.0,0.662849,0.662849,0.662849,0.662849,0.662849
2,Mick LaSalle,Lady in the Water,3.0,0.924473,0.924473,0.924473,0.924473,0.924473
3,Mick LaSalle,Just My Luck,2.0,0.924473,0.924473,0.924473,0.924473,0.924473
4,Mick LaSalle,The Night Listener,3.0,0.924473,0.924473,0.924473,0.924473,0.924473
5,Claudia Puig,Just My Luck,3.0,0.893405,0.893405,0.893405,0.893405,0.893405
6,Claudia Puig,The Night Listener,4.5,0.893405,0.893405,0.893405,0.893405,0.893405
7,Lisa Rose,Lady in the Water,2.5,0.991241,0.991241,0.991241,0.991241,0.991241
8,Lisa Rose,Just My Luck,3.0,0.991241,0.991241,0.991241,0.991241,0.991241
9,Lisa Rose,The Night Listener,3.0,0.991241,0.991241,0.991241,0.991241,0.991241


In [30]:
# 6. 영화 추천 예측
rating_t["sim_rating"] = rating_t.rating * rating_t.Toby
rating_t.head()

,critic,title,rating,Toby_x,Toby_y,Toby_x,Toby_y,Toby,sim_rating
0,Jack Matthews,Lady in the Water,3.0,0.662849,0.662849,0.662849,0.662849,0.662849,1.988547
1,Jack Matthews,The Night Listener,3.0,0.662849,0.662849,0.662849,0.662849,0.662849,1.988547
2,Mick LaSalle,Lady in the Water,3.0,0.924473,0.924473,0.924473,0.924473,0.924473,2.773420
3,Mick LaSalle,Just My Luck,2.0,0.924473,0.924473,0.924473,0.924473,0.924473,1.848947
4,Mick LaSalle,The Night Listener,3.0,0.924473,0.924473,0.924473,0.924473,0.924473,2.773420


In [32]:
#  영화별 그룹호 -> 합계
rating_g = rating_t.groupby("title")
rating_g_sum=rating_g.sum()
rating_g_sum

,rating,Toby_x,Toby_y,Toby_x,Toby_y,Toby,sim_rating
title,,,,,,,
Just My Luck,9.5,3.190366,3.190366,3.190366,3.190366,3.190366,8.074754
Lady in the Water,11.5,2.959810,2.959810,2.959810,2.959810,2.959810,8.383808
The Night Listener,16.5,3.853215,3.853215,3.853215,3.853215,3.853215,12.899752


In [34]:
#sim_rating/ Toby 

rating_g_sum["recomm"]=rating_g_sum.sim_rating/rating_g_sum.Toby
rating_g_sum

,rating,Toby_x,Toby_y,Toby_x,Toby_y,Toby,sim_rating,recomm
title,,,,,,,,
Just My Luck,9.5,3.190366,3.190366,3.190366,3.190366,3.190366,8.074754,2.530981
Lady in the Water,11.5,2.959810,2.959810,2.959810,2.959810,2.959810,8.383808,2.832550
The Night Listener,16.5,3.853215,3.853215,3.853215,3.853215,3.853215,12.899752,3.347790
